In [1]:
!pip install tqdm

In [2]:
import os
import json
from glob import glob
import xml.etree.ElementTree as ET
from PIL import Image
from sklearn.model_selection import train_test_split
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Force GPU 0 (NVIDIA)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


print("CUDA Available:", torch.cuda.is_available())
print("Device Count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current Device:", torch.cuda.current_device())
    print("Device Name:", torch.cuda.get_device_name(0))
    print("Device Capability:", torch.cuda.get_device_capability(0))

Using device: cuda
CUDA Available: True
Device Count: 1
Current Device: 0
Device Name: NVIDIA GeForce RTX 4060 Laptop GPU
Device Capability: (8, 9)


In [3]:
# Path configuration
BASE_DIR = os.path.abspath(".")
OPENLOGO_DIR = os.path.join(BASE_DIR, "openlogo")

JPEG_DIR = os.path.join(OPENLOGO_DIR, "JPEGImages")
ANNO_DIR = os.path.join(OPENLOGO_DIR, "Annotations")
IMAGESET_DIR = os.path.join(OPENLOGO_DIR, "ImageSets", "class_sep")

print("BASE_DIR:     ", BASE_DIR)
print("OPENLOGO_DIR: ", OPENLOGO_DIR)
print("JPEG_DIR:     ", JPEG_DIR)
print("ANNO_DIR:     ", ANNO_DIR)
print("IMAGESET_DIR: ", IMAGESET_DIR)

# Sanity check
some_imgs = glob(os.path.join(JPEG_DIR, "*.jpg"))[:5]
some_annos = glob(os.path.join(ANNO_DIR, "*.xml"))[:5]
print("\nSample images:", some_imgs[:2])
print("Sample annos: ", some_annos[:2])

BASE_DIR:      C:\Users\Srihari\ENPM703FinalProject
OPENLOGO_DIR:  C:\Users\Srihari\ENPM703FinalProject\openlogo
JPEG_DIR:      C:\Users\Srihari\ENPM703FinalProject\openlogo\JPEGImages
ANNO_DIR:      C:\Users\Srihari\ENPM703FinalProject\openlogo\Annotations
IMAGESET_DIR:  C:\Users\Srihari\ENPM703FinalProject\openlogo\ImageSets\class_sep

Sample images: ['C:\\Users\\Srihari\\ENPM703FinalProject\\openlogo\\JPEGImages\\1008198576.jpg', 'C:\\Users\\Srihari\\ENPM703FinalProject\\openlogo\\JPEGImages\\1016381746.jpg']
Sample annos:  ['C:\\Users\\Srihari\\ENPM703FinalProject\\openlogo\\Annotations\\1008198576.xml', 'C:\\Users\\Srihari\\ENPM703FinalProject\\openlogo\\Annotations\\1016381746.xml']


In [4]:
def parse_voc_xml(xml_path):
    """
    Parse one XML annotation. Return a list of objects.
    Each object is: {'class_name': str, 'bbox': [xmin, ymin, xmax, ymax]}
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    objects = []
    for obj in root.findall("object"):
        cls_name = obj.find("name").text.strip()

        bbox_node = obj.find("bndbox")
        xmin = int(float(bbox_node.find("xmin").text))
        ymin = int(float(bbox_node.find("ymin").text))
        xmax = int(float(bbox_node.find("xmax").text))
        ymax = int(float(bbox_node.find("ymax").text))

        objects.append({
            "class_name": cls_name,
            "bbox": [xmin, ymin, xmax, ymax]
        })

    return objects

# Test
if len(some_annos) > 0:
    test_xml = some_annos[0]
    print("Testing parse_voc_xml on:", test_xml)
    print(parse_voc_xml(test_xml)[:3])

Testing parse_voc_xml on: C:\Users\Srihari\ENPM703FinalProject\openlogo\Annotations\1008198576.xml
[{'class_name': 'guinness', 'bbox': [333, 238, 595, 414]}]


In [5]:
def build_logo_samples(imageset_dir, jpeg_dir, anno_dir, 
                       train_suffix="_train.txt", test_size=0.2, random_state=42):
    """
    Build train/val samples from OpenLogo dataset.
    Returns: train_samples, val_samples, class_to_idx
    """
    all_samples = []

    list_files = glob(os.path.join(imageset_dir, f"*{train_suffix}"))
    list_files.sort()
    print("Found train list files:", len(list_files))

    for lf in list_files:
        with open(lf, "r") as f:
            ids = [line.strip() for line in f if len(line.strip()) > 0]

        for img_id in ids:
            img_path = os.path.join(jpeg_dir, img_id + ".jpg")
            xml_path = os.path.join(anno_dir, img_id + ".xml")

            if not (os.path.exists(img_path) and os.path.exists(xml_path)):
                continue

            objs = parse_voc_xml(xml_path)

            for obj in objs:
                cls_name = obj["class_name"]
                bbox = obj["bbox"]

                all_samples.append({
                    "img_path": img_path,
                    "bbox": bbox,
                    "class_name": cls_name
                })

    # Build class mapping
    classes = sorted(list({s["class_name"] for s in all_samples}))
    class_to_idx = {c: i for i, c in enumerate(classes)}
    print("Discovered classes:", len(class_to_idx))

    # Add numeric labels
    for s in all_samples:
        s["label"] = class_to_idx[s["class_name"]]

    # Stratified split
    y = [s["label"] for s in all_samples]
    train_s, val_s = train_test_split(
        all_samples,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )

    print(f"Total samples: {len(all_samples)} -> train {len(train_s)}, val {len(val_s)}")
    return train_s, val_s, class_to_idx


train_samples, val_samples, class_to_idx = build_logo_samples(
    IMAGESET_DIR, JPEG_DIR, ANNO_DIR,
    train_suffix="_train.txt", test_size=0.2, random_state=42
)

num_classes = len(class_to_idx)
print("num_classes:", num_classes)
print("\nSample:", train_samples[0])

Found train list files: 353
Discovered classes: 352
Total samples: 72652 -> train 58121, val 14531
num_classes: 352

Sample: {'img_path': 'C:\\Users\\Srihari\\ENPM703FinalProject\\openlogo\\JPEGImages\\CPA_Australia_sportslogo_22.jpg', 'bbox': [1, 330, 54, 352], 'class_name': 'cpa_australia', 'label': 104}


In [6]:
class LogoCropDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        path = s["img_path"]
        xmin, ymin, xmax, ymax = s["bbox"]
        label = s["label"]

        # Load image
        img = Image.open(path).convert("RGB")

        # Defensive clamp
        w, h = img.size
        xmin_c = max(0, min(xmin, w - 1))
        ymin_c = max(0, min(ymin, h - 1))
        xmax_c = max(0, min(xmax, w))
        ymax_c = max(0, min(ymax, h))

        if xmax_c <= xmin_c or ymax_c <= ymin_c:
            crop = img
        else:
            crop = img.crop((xmin_c, ymin_c, xmax_c, ymax_c))

        if self.transform is not None:
            crop = self.transform(crop)

        return crop, label

In [7]:
train_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_ds = LogoCropDataset(train_samples, transform=train_transform)
val_ds = LogoCropDataset(val_samples, transform=val_transform)

BATCH_SIZE = 32

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=(device.type == "cuda")
)

val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=(device.type == "cuda")
)

print("Train batches:", len(train_loader), "Val batches:", len(val_loader))

Train batches: 1817 Val batches: 455


In [8]:
import torch.nn.functional as F

class CustomCNN_Logo(nn.Module):
    """
    Custom CNN that WORKS - simplified but effective architecture
    """
    def __init__(self, num_classes=352, dropout=0.3):
        super().__init__()
        
        # Conv Block 1
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 224 -> 112
        )
        
        # Conv Block 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 112 -> 56
        )
        
        # Conv Block 3
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 56 -> 28
        )
        
        # Conv Block 4
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 28 -> 14
        )
        
        # Global Average Pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


print("="*60)
print("Initializing Custom CNN for Logo Classification")
print("="*60)

model = CustomCNN_Logo(num_classes=num_classes, dropout=0.3).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / 1024 / 1024:.2f} MB")
print("\n" + "="*60)

Initializing Custom CNN for Logo Classification

Total parameters: 8,083,872
Trainable parameters: 8,083,872
Model size: ~30.84 MB



In [9]:
criterion = nn.CrossEntropyLoss()

# Higher initial learning rate for faster convergence
optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

print("Optimizer: Adam")
print("Initial Learning Rate: 3e-4")
print("Weight Decay: 1e-4")
print("LR Scheduler: StepLR (decay by 0.5 every 5 epochs)")

Optimizer: Adam
Initial Learning Rate: 3e-4
Weight Decay: 1e-4
LR Scheduler: StepLR (decay by 0.5 every 5 epochs)


In [10]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_correct = 0
    running_total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        running_correct += (preds == labels).sum().item()
        running_total += labels.size(0)

    epoch_loss = running_loss / running_total
    epoch_acc = running_correct / running_total
    return epoch_loss, epoch_acc


@torch.no_grad()
def eval_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_correct = 0
    running_total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        running_correct += (preds == labels).sum().item()
        running_total += labels.size(0)

    epoch_loss = running_loss / running_total
    epoch_acc = running_correct / running_total
    return epoch_loss, epoch_acc

In [11]:
import time

EPOCHS = 10  # More epochs for better accuracy

best_val_acc = 0.0
best_state_dict = None

history = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}

print(f"\nStarting training for {EPOCHS} epochs...")
print("="*60)

total_start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"{'='*60}")
    
    # TRAINING
    model.train()
    running_loss = 0.0
    running_correct = 0
    running_total = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        running_correct += (preds == labels).sum().item()
        running_total += labels.size(0)
        
        # Print progress every 200 batches
        if (batch_idx + 1) % 200 == 0:
            batch_acc = running_correct / running_total
            print(f"  [{batch_idx+1:4d}/{len(train_loader)}] "
                  f"loss: {loss.item():.4f}, acc: {batch_acc:.4f}")

    train_loss = running_loss / running_total
    train_acc = running_correct / running_total
    
    # VALIDATION
    model.eval()
    running_loss = 0.0
    running_correct = 0
    running_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            running_correct += (preds == labels).sum().item()
            running_total += labels.size(0)
    
    val_loss = running_loss / running_total
    val_acc = running_correct / running_total
    
    # Update learning rate based on validation accuracy
    scheduler.step()
    
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)
    
    epoch_time = time.time() - epoch_start_time
    
    print(f"\n{'='*60}")
    print(f"[Epoch {epoch+1}/{EPOCHS} Summary]")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
    print(f"  Time: {epoch_time:.1f}s")
    print(f"  Current LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}
        print(f"  New BEST model! (val_acc={best_val_acc:.4f})")
    print(f"{'='*60}")

total_time = time.time() - total_start_time

print(f"\n{'='*60}")
print("=== Training Complete ===")
print(f"{'='*60}")
print(f"Best Validation Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
print(f"Total Training Time: {total_time/60:.1f} minutes")
print(f"Average Time per Epoch: {total_time/EPOCHS:.1f} seconds")


Starting training for 10 epochs...

Epoch 1/10
  [ 200/1817] loss: 4.8026, acc: 0.0533
  [ 400/1817] loss: 4.1379, acc: 0.0631
  [ 600/1817] loss: 4.5574, acc: 0.0723
  [ 800/1817] loss: 5.0171, acc: 0.0811
  [1000/1817] loss: 4.4030, acc: 0.0882
  [1200/1817] loss: 3.9808, acc: 0.0955
  [1400/1817] loss: 3.6371, acc: 0.1037
  [1600/1817] loss: 4.1331, acc: 0.1131
  [1800/1817] loss: 4.2470, acc: 0.1216

[Epoch 1/10 Summary]
  Train Loss: 4.4316 | Train Acc: 0.1224
  Val Loss:   3.8099 | Val Acc:   0.2081
  Time: 1338.6s
  Current LR: 0.000300
  New BEST model! (val_acc=0.2081)

Epoch 2/10
  [ 200/1817] loss: 4.4671, acc: 0.2162
  [ 400/1817] loss: 3.7377, acc: 0.2200
  [ 600/1817] loss: 2.9227, acc: 0.2311
  [ 800/1817] loss: 3.2133, acc: 0.2399
  [1000/1817] loss: 3.2529, acc: 0.2503
  [1200/1817] loss: 3.2759, acc: 0.2624
  [1400/1817] loss: 3.0216, acc: 0.2690
  [1600/1817] loss: 2.8207, acc: 0.2759
  [1800/1817] loss: 3.4289, acc: 0.2819

[Epoch 2/10 Summary]
  Train Loss: 3.3780

In [13]:
if best_state_dict is not None:
    torch.save(best_state_dict, "best_custom_cnn_image_custom.pth")
    print("Saved best model -> best_custom_cnn_image_custom.pth")

with open("class_to_idx_image_custom.json", "w") as f:
    json.dump(class_to_idx, f, indent=2)
print("Saved class_to_idx_image_custom.json")

print("\nTraining curves:")
for i in range(EPOCHS):
    print(
        f"Epoch {i+1:02d}: "
        f"train_acc={history['train_acc'][i]:.4f}, "
        f"val_acc={history['val_acc'][i]:.4f}, "
        f"train_loss={history['train_loss'][i]:.4f}, "
        f"val_loss={history['val_loss'][i]:.4f}"
    )

Saved best model -> best_custom_cnn_image_custom.pth
Saved class_to_idx_image_custom.json

Training curves:
Epoch 01: train_acc=0.1224, val_acc=0.2081, train_loss=4.4316, val_loss=3.8099
Epoch 02: train_acc=0.2823, val_acc=0.3824, train_loss=3.3780, val_loss=2.7908
Epoch 03: train_acc=0.3958, val_acc=0.4727, train_loss=2.7155, val_loss=2.3092
Epoch 04: train_acc=0.4814, val_acc=0.5641, train_loss=2.2738, val_loss=1.9102
Epoch 05: train_acc=0.5487, val_acc=0.6121, train_loss=1.9397, val_loss=1.6334
Epoch 06: train_acc=0.6255, val_acc=0.6922, train_loss=1.5827, val_loss=1.3089
Epoch 07: train_acc=0.6611, val_acc=0.7251, train_loss=1.4212, val_loss=1.1691
Epoch 08: train_acc=0.6901, val_acc=0.7471, train_loss=1.2916, val_loss=1.0680
Epoch 09: train_acc=0.7117, val_acc=0.7503, train_loss=1.1840, val_loss=1.0529
Epoch 10: train_acc=0.7356, val_acc=0.7630, train_loss=1.0859, val_loss=0.9793
